To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [1]:
print("yoloooo test that shheet neeega")

yoloooo test that shheet neeega


In [2]:
import torch
print(torch.cuda.is_available())           # Doit être True
print(torch.cuda.get_device_name(0))       # Doit afficher "NVIDIA GeForce RTX 3070 Ti"

True
NVIDIA GeForce RTX 3070 Ti


### Unsloth

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2b-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.7.0+cu126 with CUDA 1206 (you have 2.7.0+cu118)
    Python  3.12.10 (you have 3.12.9)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Gemma patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 3070 Ti. Num GPUs = 1. Max memory: 8.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.7.0+cu118. CUDA: 8.6. CUDA Toolkit: 11.8. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.4.7 patched 18 layers with 18 QKV layers, 18 O layers and 18 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `ChatML` format for conversation style finetunes. We use [Open Assistant conversations](https://huggingface.co/datasets/philschmid/guanaco-sharegpt-style) in ShareGPT style. ChatML renders multi turn conversations like below:

```
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What's the capital of France?<|im_end|>
<|im_start|>assistant
Paris.
```

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old` and our own optimized `unsloth` template.

Normally one has to train `<|im_start|>` and `<|im_end|>`. We instead map `<|im_end|>` to be the EOS token, and leave `<|im_start|>` as is. This requires no additional training of additional tokens.

Note ShareGPT uses `{"from": "human", "value" : "Hi"}` and not `{"role": "user", "content" : "Hi"}`, so we use `mapping` to map it.

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("philschmid/guanaco-sharegpt-style", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)
# ultrachat_train = df

README.md:   0%|          | 0.00/442 [00:00<?, ?B/s]

(…)-00000-of-00001-8aae24b47ddaaf21.parquet:   0%|          | 0.00/8.24M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9033 [00:00<?, ? examples/s]

Map:   0%|          | 0/9033 [00:00<?, ? examples/s]

In [23]:
# For google collab only
from google.colab import files
import pandas as pd

# Télécharger les fichiers depuis l'ordinateur local
eval_dataset = files.upload()

# Lire les fichiers JSONL téléchargés
for filename in eval_dataset.keys():
    if filename.endswith('.jsonl'):
        df = pd.read_json(filename, lines=True)
        print(f"Data from {filename} loaded successfully")
        print(df.head())


Saving ultrachat_second_selection_training_dataset.jsonl to ultrachat_second_selection_training_dataset (1).jsonl
Data from ultrachat_second_selection_training_dataset (1).jsonl loaded successfully
                                            messages
0  [{'role': 'user', 'content': '{
  "Chapter dat...
1  [{'role': 'user', 'content': '{
  "Chapter dat...
2  [{'role': 'user', 'content': '{
  "Chapter dat...
3  [{'role': 'user', 'content': '{
  "Chapter dat...
4  [{'role': 'user', 'content': '{
  "Chapter dat...


In [ ]:
# For local usage only
import pandas as pd
ultrachat_train = pd.read_json(r'C:\Users\amande\Desktop\Pierre\TI_github\Terra_Inco\data\ultrachat_second_selection_training_dataset.jsonl', lines=True)

In [ ]:
ultrachat_train = ultrachat_train.map(formatting_prompts_func, batched = True,)

Let's see how the `ChatML` format works by printing the 5th element

In [34]:
import numpy as np
#print(np.shape(dataset))
print(np.shape(ultrachat_train))

(32, 1)


In [10]:
ultrachat_train

,messages
0,"[{'role': 'user', 'content': '{ ""Chapter dat..."
1,"[{'role': 'user', 'content': '{ ""Chapter dat..."
2,"[{'role': 'user', 'content': '{ ""Chapter dat..."
3,"[{'role': 'user', 'content': '{ ""Chapter dat..."
4,"[{'role': 'user', 'content': '{ ""Chapter dat..."
5,"[{'role': 'user', 'content': '{ ""Chapter dat..."
6,"[{'role': 'user', 'content': '{ ""Chapter dat..."
7,"[{'role': 'user', 'content': '{ ""Chapter dat..."
8,"[{'role': 'user', 'content': '{ ""Chapter dat..."
9,"[{'role': 'user', 'content': '{ ""Chapter dat..."


In [11]:
# Explode the 'messages' column to unpack the lists of dictionaries
exploded_df = ultrachat_train.explode('messages').reset_index()

# Normalize the 'messages' column to create separate columns for 'role' and 'content'
normalized_df = pd.json_normalize(exploded_df['messages'])

# Add the original index as a column to group by later
normalized_df['index'] = exploded_df['index']

# Pivot the DataFrame to create separate columns for 'user' and 'assistant'
pivoted_df = normalized_df.pivot(index='index', columns='role', values='content').reset_index(drop=True)

# Display the first few rows of the transformed DataFrame
print(pivoted_df.head())

role                                          assistant  \
0     [Sidenote: A Counterblaste to James I.]\n\nSo ...   
1     XXXVI\n\nAN ALLY, AND SOLID GROUND\n\n\nThe so...   
2     The Selenite's Face\n\nI found myself sitting ...   
3     About a month after the perfidious Werther's w...   
4     CHAPTER XIX\n\nA FIENDISH MACHINE\n\n\nA chill...   

role                                               user  
0     {\n  "Chapter data": {\n    "Book": 9468,\n   ...  
1     {\n  "Chapter data": {\n    "Book": 9692,\n   ...  
2     {\n  "Chapter data": {\n    "Book": 53,\n    "...  
3     {\n  "Chapter data": {\n    "Book": 1088,\n   ...  
4     {\n  "Chapter data": {\n    "Book": 2971,\n   ...  


In [12]:
print(ultrachat_train.head())
print(ultrachat_train.columns)

                                            messages
0  [{'role': 'user', 'content': '{
  "Chapter dat...
1  [{'role': 'user', 'content': '{
  "Chapter dat...
2  [{'role': 'user', 'content': '{
  "Chapter dat...
3  [{'role': 'user', 'content': '{
  "Chapter dat...
4  [{'role': 'user', 'content': '{
  "Chapter dat...
Index(['messages'], dtype='object')


In [13]:
pivoted_df

role,assistant,user
0,[Sidenote: A Counterblaste to James I.]\n\nSo ...,"{\n ""Chapter data"": {\n ""Book"": 9468,\n ..."
1,"XXXVI\n\nAN ALLY, AND SOLID GROUND\n\n\nThe so...","{\n ""Chapter data"": {\n ""Book"": 9692,\n ..."
2,The Selenite's Face\n\nI found myself sitting ...,"{\n ""Chapter data"": {\n ""Book"": 53,\n ""..."
3,About a month after the perfidious Werther's w...,"{\n ""Chapter data"": {\n ""Book"": 1088,\n ..."
4,CHAPTER XIX\n\nA FIENDISH MACHINE\n\n\nA chill...,"{\n ""Chapter data"": {\n ""Book"": 2971,\n ..."
5,"THE HEARTBREAK CAMEO.\n\n\n""It is a cameo to b...","{\n ""Chapter data"": {\n ""Book"": 7627,\n ..."
6,"CHAPTER XVI\n\nMRS. VRAIN AT BAY\n\n\n""You saw...","{\n ""Chapter data"": {\n ""Book"": 3398,\n ..."
7,CHAPTER X\n\nDOWN A BIG HOLE\n\n\nFor a few se...,"{\n ""Chapter data"": {\n ""Book"": 2498,\n ..."
8,CHAPTER IX\n\nMR. THOMAS MARVEL\n\n\nYou must ...,"{\n ""Chapter data"": {\n ""Book"": 8462,\n ..."
9,"JOHN WESLEY.\n\n""_THE DIVISION OF THE CHURCH O...","{\n ""Chapter data"": {\n ""Book"": 1228,\n ..."


In [14]:
print(pivoted_df["assistant"])
print(pivoted_df["user"])

0     [Sidenote: A Counterblaste to James I.]\n\nSo ...
1     XXXVI\n\nAN ALLY, AND SOLID GROUND\n\n\nThe so...
2     The Selenite's Face\n\nI found myself sitting ...
3     About a month after the perfidious Werther's w...
4     CHAPTER XIX\n\nA FIENDISH MACHINE\n\n\nA chill...
5     THE HEARTBREAK CAMEO.\n\n\n"It is a cameo to b...
6     CHAPTER XVI\n\nMRS. VRAIN AT BAY\n\n\n"You saw...
7     CHAPTER X\n\nDOWN A BIG HOLE\n\n\nFor a few se...
8     CHAPTER IX\n\nMR. THOMAS MARVEL\n\n\nYou must ...
9     JOHN WESLEY.\n\n"_THE DIVISION OF THE CHURCH O...
10    CHAPTER V\n\nCantrell's Comet\n\n\nFar out in ...
11    XIV\n\nNext day I fell ill. I had felt unwell ...
12    _XVII_\n\nTHE SLOWING ROTATION\n\nIt might hav...
13    CHAPTER XXVIII\n\nGeorgiana Ford would have ha...
14    VI.\n\n\nAfter that they were together constan...
15    II\n\nTHE ORIGIN OF ENERGY\n\n\nI\n\nWhat has ...
16    CHAPTER XI\n\nDEAR MAMMA\n\n\nBERYL must have ...
17    CHAPTER XXVIII. THE FRENZY OF RUTH\n\nFor 

In [31]:
print(np.shape(dataset))
print(type(dataset))
print(np.shape(pivoted_df))
print(type(pivoted_df))

(9033, 2)
<class 'datasets.arrow_dataset.Dataset'>
(32, 2)
<class 'pandas.core.frame.DataFrame'>


In [30]:
dataset[0]

{'conversations': [{'from': 'human',
   'value': 'Escribe un discurso que pueda recitar como padrino de la boda de mi mejor amigo.'},
  {'from': 'gpt',
   'value': 'Queridos invitados, amigos y familiares,\n\nMe siento muy honrado de estar aquí hoy como padrino de bodas de mi mejor amigo [Nombre del novio].\n\nRecuerdo con cariño los días en los que [Nombre del novio] y yo nos conocimos, cuando éramos solo dos jóvenes llenos de sueños y esperanza. Ahora, aquí estamos, celebrando su boda con la persona que ama, [Nombre de la novia].\n\n[Nombre de la novia], te aseguro que [Nombre del novio] es una persona increíble, llena de amor y lealtad. Juntos, han formado un equipo invencible y estoy seguro de que su amor perdurará por siempre.\n\n[Nombre del novio], mi amigo, te deseo todo lo mejor en esta nueva etapa de tu vida. Espero que tu matrimonio esté lleno de amor, alegría y felicidad, y que [Nombre de la novia] sea siempre tu compañera de vida y tu mejor amiga.\n\nA [Nombre de la novia],

In [29]:
pivoted_df[0]

KeyError: 0

If you're looking to make your own chat template, that also is possible! You must use the Jinja templating regime. We provide our own stripped down version of the `Unsloth template` which we find to be more efficient, and leverages ChatML, Zephyr and Alpaca styles.

More info on chat templates on [our wiki page!](https://github.com/unslothai/unsloth/wiki#chat-templates)

In [51]:
unsloth_template = \
    "{{ bos_token }}"\
    "{{ 'You are a helpful assistant to the user\n' }}"\
    "{% for message in messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ '>>> User: ' + message['content'] + '\n' }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ '>>> Assistant: ' + message['content'] + eos_token + '\n' }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '>>> Assistant: ' }}"\
    "{% endif %}"
unsloth_eos_token = "eos_token"

if False:
    tokenizer = get_chat_template(
        tokenizer,
        chat_template = (unsloth_template, unsloth_eos_token,), # You must provide a template and EOS token
        mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
        map_eos_token = True, # Maps <|im_end|> to </s> instead
    )

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [17]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = pivoted_df,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

KeyError: 0

In [53]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
7.0 GB of memory reserved.


In [54]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,033 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/7,000,000,000 (0.60% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.636300
2,1.158100
3,0.975700
4,1.099400
5,0.980900
6,1.022100
7,0.824800
8,1.186800
9,1.187200
10,1.058100


In [55]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

888.1694 seconds used for training.
14.8 minutes used for training.
Peak reserved memory = 7.0 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 47.487 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! Since we're using `ChatML`, use `apply_chat_template` with `add_generation_prompt` set to `True` for inference.

In [56]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|im_start|>user\nContinue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|im_end|>\n<|im_start|>assistant\nThe next number in the Fibonacci sequence is 13. This is calculated by adding the last two numbers in the sequence together: 8 + 5 = 13. The Fibonacci sequence continues in this way, with each number being the sum of the two numbers before it.<|im_end|>']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [57]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

<|im_start|>user
Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|im_end|>
<|im_start|>assistant
The next number in the Fibonacci sequence is 13. This is calculated by adding the last two numbers in the sequence together: 8 + 5 = 13. The Fibonacci sequence continues in this way, with each number being the sum of the two numbers before it.<|im_end|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [58]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [59]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "What is a famous tall tower in Paris?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

<|im_start|>user
What is a famous tall tower in Paris?<|im_end|>
<|im_start|>assistant
The Eiffel Tower is a famous tall tower in Paris. It is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower. The Eiffel Tower is one of the most recognizable structures in the world and is a popular tourist destination. It is also a symbol of France and is often used as a landmark for navigation.<|im_end|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoModelForPeftCausalLM
    from transformers import AutoTokenizer

    model = AutoModelForPeftCausalLM.from_pretrained(
        "lora_model",  # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit=load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
